# Inferencja w generatorze RNN przy użyciu GPU lub CPU

In [0]:
from pathlib import Path
dataset_path = Path.home() / 'data/rnn_generator'; dataset_path
tmp_path = dataset_path / 'tmp/'

In [2]:
import torch

USE_GPU = torch.cuda.is_available(); 
# USE_GPU = False; 

print(f'USE_GPU={USE_GPU}')

def to_gpu(x, *args, **kwargs):
    return x.cuda(*args, **kwargs) if USE_GPU else x

USE_GPU=True


In [0]:
ALLCHARS, MODEL = ['all_characters', 'model']

fn_witkacy = {ALLCHARS: 'all_characters.p', MODEL: 'model.h100.l1.e2000.cpu.dat'}
# fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e2000.cpu.torch'}
# fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h100.l1.e101.gpu.torch'}
fn_pan_tadeusz = {ALLCHARS: 'all_characters.pan_tadeusz.p', MODEL: 'pan_tadeusz.h400.l3.3000.gpu.torch'}


fn_dict = fn_pan_tadeusz

In [4]:
tmp_path

PosixPath('/content/data/rnn_generator/tmp')

In [5]:
ls -lah $tmp_path

total 8.5M
drwxr-xr-x 2 root root 4.0K May  8 16:02 ./
drwxr-xr-x 3 root root 4.0K May  8 13:19 ../
-rw-r--r-- 1 root root  743 May  8 16:02 all_characters.pan_tadeusz.p
-rw-r--r-- 1 root root 8.5M May  8 16:02 pan_tadeusz.h400.l3.3000.gpu.torch


## Ładowanie listy all_characters

In [6]:
import pickle

all_characters = pickle.load( open( tmp_path / fn_dict[ALLCHARS], 'rb' ) ); print(all_characters[:10])
n_characters = len(all_characters); print(n_characters)

['\n', ' ', '!', '"', '%', "'", '(', ')', ',', '-']
89


## Kod

In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        input = self.encoder(input.view(1, -1))
        output, hidden = self.gru(input.view(1, 1, -1), hidden)
        output = self.decoder(output.view(1, -1))
        return output, hidden

    def init_hidden(self):
        return Variable(to_gpu(torch.zeros(self.n_layers, 1, self.hidden_size)))

In [8]:
# Turn string into list of longs
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return Variable(to_gpu(tensor))

print(char_tensor('ala ma kota'))

Variable containing:
 41
 52
 41
  1
 53
 41
  1
 51
 55
 59
 41
[torch.cuda.LongTensor of size 11 (GPU 0)]



In [0]:
def evaluate(prime_str='A', predict_len=100, temperature=0.8):
    hidden = decoder.init_hidden()
    prime_input = char_tensor(prime_str)
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[p], hidden)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char)

    return predicted

In [10]:
n_epochs = 300 # 3000
print_every = 100
plot_every = 1
hidden_size = 100
n_layers = 1
lr = 0.005

decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
print(decoder, flush=True)

RNN(
  (encoder): Embedding(89, 100)
  (gru): GRU(100, 100)
  (decoder): Linear(in_features=100, out_features=89)
)


## Wczytywanie modelu

In [11]:
model_path_cpu = tmp_path / fn_dict[MODEL]
decoder = torch.load(model_path_cpu)
print(decoder, flush=True)

RNN(
  (encoder): Embedding(89, 300)
  (gru): GRU(300, 300, num_layers=4)
  (decoder): Linear(in_features=300, out_features=89)
)


In [0]:
decoder.gru.flatten_parameters()

## Generowanie tekstu

In [0]:
# priming strings
# pan_tadeusz: kon, bę, Tad, Tadeusz

In [14]:
print(evaluate('Tadeusz', 1000, temperature=0.8))

Tadeusz niego s dzima s czy pięcy.


Wolich twrzelém,
I stani w do korzanbie.


Tarunicz czwéj
Na się o kodziona po w balą w s czystaną stowałka mogał jasić żalo, dzi zsię, postawa, na się sią udzico rzet czarta pod wszysich się powanie do zusła zielżą daze nadzy kosta naraz swana z biędzigą ukał,
Tam zą nież jtak znesznia spowada poczy na muszela,
Rzepy, zagopadzać żaga,
Doznie zanął sani niegch garz sza, rzy zusza ogastech do buży ubał nan sprzykli
Ddoweby siel ocz udziości bachzął;
Wypnionaniem.

Zawanie zwaraz postrzeszać poz pan zaloszanie ma znierząc stadził.

Tu strzyskrowo przyszczyźnu nal dobie nopaślu ja zmostu,
«Apasi grzeniwać w lade z żylku oszego upo naganowy do kranik,
Dłurzyk go dostali sywyszciém ciet teże, pon polkia lachniem niby jak  go przennny;
Wy loszczych z wzyn buka że --
A roszieł wszykać wiego w nie sunnia s ba pana,
Czydzia,
Redzieni naranie Gadzi, w w jak w pogować jaszy
W usta zasił doże  pobanie, pogał puszy!
I się płowiemczynie, Coże poru na nie banie ni

In [15]:
print(evaluate('Tad', 1000, temperature=0.5))

Tadziem w rosta miele się się zbosta trzy za krzeszczynie go z w kodzukł bary się; stana się wieszyskać szanie w wz czyszczele;
Rawarząd powielano:
Tadzie w ponia,
Cak się w tak puszki w rabiem spowanie postach w kola jak cz czartawać panie nieży po czanie posta się obaranie szeszył spostanie charaz na na s w w i sposkał sponie cać na usta spodzieli jaczawieni nie postani do był z za czawił jak stania wiele się jak, na z zanie przać obalki,
Posta sprzyszył się się growieno częsty poszczy nie jesze stadzia od w sabym się w się i sieskie uskło chowie obył bary się krzel w cie się na marszczy stawie się s dale komiania postał krowska o czyski stawie z wieszczania z wnie,
Po wszeski w suby,
Dosta pochu dosko jak tak odwieszcze z na koraznowiem s siedzie obarał się ma pyle stadzie piędziejo w kroducho pod w grzelanie s pobozuwie podwanie,
Na gowi się posz z wszysze mał się obarawa,
A sta pod i się wsta spony,
Jest na poganie na na głowiem,
Do zabie się poli pieposzyć spowawie na się się się

In [16]:
print(evaluate('bę', 200, temperature=1.4))

bęmiém zoły przeżówsz kojsko weknóżny
Ota nibuwsło wiy!
Apokugo Wozawiwa neszcia_ i rzyja dżajem,
Ka.
ĘMieją, datcoje ż żawery,
Agki okyć na wssapokłastych zażę:
Udszesorzęcby
Kdzas dzieniéj w zuskowszy
